In [25]:
include("LR_Pred_ST.jl")
Random.seed!(124)

TaskLocalRNG()

In [26]:
h = 0.2; x0 = -1.0; xn = 1.0; activation = relu; dim = 1; N1 = 10_000
# We create some data (normalized)
x = hcat(range(x0,stop=xn,step=h)...)
f = x->sin(5*x)+cos(5*x)

model = Chain(DenseNTK(dim=>N1,activation),DenseNTK(N1=>dim))|>f64
detail = 3
Iterations = 100000
println("Approximately 15 sec delay do to kernel calculations")
LR_opt2 = 1/LR_updt(h,x0,xn,N1,detail)

Loss(model,x,y) = Flux.mse(y,model(x))

Approximately 15 sec delay do to kernel calculations


UndefVarError: UndefVarError: `iterations` not defined

In [27]:

L = []
y = map(f,x)
data = [(x,y)]
@showprogress for epoch = 1:Iterations

    Flux.train!(Loss,model,data,Descent(LR_opt2))
    push!(L,Loss(model,x,y))

end 

Progress:  22%|█████████                                |  ETA: 0:01:55

In [ ]:
L